# Step 6: Add data monitoring
After executing five previous notebooks, you have a production-ready solution with automated model building and model deployment CI/CD pipelines.
This notebook adds continuous monitoring of the data quality in real-time. [Amazon SageMaker model monitor](https://aws.amazon.com/sagemaker/model-monitor/) enables you to set up an automated alert triggering system when there are deviations in the data and model quality, such as data drift and anomalies.

![](img/six-steps-6.png)

In [ ]:
%pip install jsonlines

In [ ]:
import boto3
import sagemaker 
import json
import jsonlines
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.predictor import Predictor
import time
from time import gmtime, strftime
import uuid
import pandas as pd
import numpy as np
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator
from utils.monitoring_utils import run_model_monitor_job

In [ ]:
sm = boto3.client("sagemaker")
s3 = boto3.client("s3")

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

## How model monitor works
Amazon SageMaker Model Monitor automatically monitors ML models in production and notifies you when quality issues arise. Model Monitor uses rules to detect drift in your models and data and alerts you when it happens. The following figure shows how this process works.

![](img/model-monitor.png)

The process for setting up the data monitoring:
1. Enable the endpoint to capture data from incoming requests to a trained ML model and the resulting model predictions
2. Create a baseline from the dataset that was used to train the model. The baseline computes metrics and suggests constraints for the metrics. Real-time predictions from your model are compared to the constraints, and are reported as violations if they are outside the constrained values
3. Create a monitoring schedule specifying what data to collect, how often to collect it, how to analyze it, and which reports to produce
4. Inspect the reports, which compare the latest data with the baseline, and watch for any violations reported and for metrics and notifications from Amazon CloudWatch.

## Real-time inference data capture from a SageMaker endpoint
To work with the model monitor, use the existing endpoints deployed by the deployment pipeline in the step 5 notebook.

The model deployment MLOps project implemented in the step 5 notebook contains a data capture configuration for the deployed endpoints. If you clone the project's code repository to the Studio file system, you can see the project files.

The CloudFormation deployment template `endpoint-config-template.yml` enables data capture for the endpoint configuration:
```yaml
EndpointConfig:
    Type: AWS::SageMaker::EndpointConfig
    Properties:
      ProductionVariants:
        - InitialInstanceCount: !Ref EndpointInstanceCount
          InitialVariantWeight: 1.0
          InstanceType: !Ref EndpointInstanceType
          ModelName: !GetAtt Model.ModelName
          VariantName: AllTraffic
      DataCaptureConfig:
          EnableCapture: !Ref EnableDataCapture 
          InitialSamplingPercentage: !Ref SamplingPercentage
          DestinationS3Uri: !Ref DataCaptureUploadPath
          CaptureOptions:
            - CaptureMode: Input
            - CaptureMode: Output
          CaptureContentTypeHeader:
            CsvContentTypes:
              - "text/csv"
```

The configuration files `prod-config.json` and `staging-config.json` provide the actual values for `EnableCapture`, `InitialSamplingPercentage`, and `DestinationS3Uri`:
```json
{
  "Parameters": {
    "StageName": "prod",
    "EndpointInstanceCount": "1",
    "EndpointInstanceType": "ml.m5.large",
    "SamplingPercentage": "80",
    "EnableDataCapture": "true"
  }
}
```

Let's check the endpoint configuration and see how data capture is confgured.

In [ ]:
for ep in sm.list_endpoints(StatusEquals="InService")["Endpoints"]:
    print(f"Data capture configuration for {ep['EndpointName']}:")
    print(f"{json.dumps(sm.describe_endpoint(EndpointName=ep['EndpointName'])['DataCaptureConfig'], indent=2)}")

In [ ]:
# Get the configuration for a specific endpoint name
endpoint_name = "model-deploy-30-21-35-29-staging" # USE YOUR ENDPOINT NAME
data_capture_s3_url = sm.describe_endpoint(EndpointName=endpoint_name)['DataCaptureConfig']['DestinationS3Uri']
data_capture_bucket = data_capture_s3_url.split('/')[2]
data_capture_prefix = '/'.join(data_capture_s3_url.split('/')[3:])

### Define helper functions
Define some helper functions with code snippets that you're going to use throughout this notebook.

In [ ]:
# Print progress bar
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
# Send data to the endpoint
def generate_endpoint_traffic(predictor, data):
    l = len(data)
    print(f"Sending {l} vectors to the endpoint")
    for i in range(0,l):
        predictions = np.array(predictor.predict(data.iloc[i].values), dtype=float).squeeze()
        time.sleep(0.001)
        printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

In [ ]:
# Get S3 url for the latest captured data
def get_latest_data_capture_s3_url(bucket, prefix):
    try:
        capture_files = [
            capture_file.get("Key") 
            for capture_file in s3.list_objects(Bucket=bucket, Prefix=prefix).get("Contents")
        ]

        latest_data_capture_s3_url = f"s3://{bucket}/{'/'.join(capture_files[-1].split('/')[:-1])}"

        print(f"Latest data capture S3 url: {latest_data_capture_s3_url}")
        
        return latest_data_capture_s3_url
    except TypeError:
        print(f"No data capture files found in s3://{bucket}/{prefix}")
        return None

In [ ]:
# Get S3 url for the latest monitoring job output
def get_latest_monitoring_report_s3_url(job_name):
    monitor_job = sm.list_processing_jobs(
        NameContains=job_name,
        SortOrder='Descending',
        MaxResults=2
    )['ProcessingJobSummaries'][0]['ProcessingJobName']

    monitoring_job_output_s3_url = sm.describe_processing_job(
        ProcessingJobName=monitor_job
    )['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']

    print(f"Latest monitoring report S3 url: {monitoring_job_output_s3_url}")
    
    return monitoring_job_output_s3_url

In [ ]:
# Helper to load a json file from S3
def load_json_from_file(file_s3_url):
    bucket = file_s3_url.split('/')[2]
    key = '/'.join(file_s3_url.split('/')[3:])
    print(f"Load JSON from: {bucket}/{key}")
    
    return json.loads(
        s3.get_object(Bucket=bucket, 
                      Key=key)["Body"].read().decode("utf-8")
    )

### Generate captured data
You must send some data to an endpoint for inference to generate data capture.

In [ ]:
predictor = Predictor(
    endpoint_name=endpoint_name, 
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()
)

Use test dataset prepared in the step 2 notebook (`test_x.csv`):

In [ ]:
test_x = pd.read_csv("tmp/test_x.csv", names=[f'_c{i}' for i in range(59)])

In [ ]:
test_x.head(1)

Send the data to the endpoint:

In [ ]:
%%time

generate_endpoint_traffic(predictor, test_x)

### View captured data
Now list the data capture files stored in Amazon S3. The data is stored as `jsonl` an Amazon S3 path format is `s3://{data-capture-destination-s3-url}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`.

Wait until captured data appears in the Amazon S3 bucket, it may take several minutes.

In [ ]:
!aws s3 ls {data_capture_s3_url} --recursive

In [ ]:
capture_files = [
    capture_file.get("Key") 
    for capture_file in s3.list_objects(Bucket=data_capture_bucket, Prefix=data_capture_prefix).get("Contents")
]
print("Found data capture files:")
print("\n ".join(capture_files))

In [ ]:
assert(len(capture_files) > 0)

Each inference request is captured in one line in the `jsonl` file. The line contains both the input and output merged together. In the example, you provided the ContentType as `text/csv` which is reflected in the `observedContentType` value. Also, you expose the encoding that you used to encode the input and output payloads in the capture format with the encoding value.

In [ ]:
# Download the last captured dataset to Studio's EFS
!aws s3 cp s3://{data_capture_bucket}/{capture_files[-1]} ./tmp/datacapture.jsonl

In [ ]:
# Read the jsonl object and show two first requests
with jsonlines.open('./tmp/datacapture.jsonl') as reader:      
    print(json.dumps(reader.read(), indent=2))
    print(json.dumps(reader.read(), indent=2))

## Model monitor - monitor data quality
In this example you learn how to setup data quality monitoring.

To enable inference data quality monitoring and evaluation you must:
1. Enable [data capture](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html)
1. [Create a baseline](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-create-baseline.html) with which you compare the realtime traffic
1. Once a baseline is ready, [schedule monitoring jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-scheduling.html) to continously evaluate and compare against the baseline
1. [See and interpret the results](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-results.html) of monitoring jobs
1. [Integrate data quality monitoring](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-cloudwatch.html) with Amazon CloudWatch

### Create a baselining job with training dataset
The whole dataset with which you trained and tested the model is usually a good baseline dataset. Note that the baseline dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the baseline dataset you can ask Amazon SageMaker to suggest a set of baseline _constraints_ and generate descriptive _statistics_ to explore the data. Model Monitor provides a [built-in container](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-built-container.html) that provides the ability to suggest the constraints automatically for CSV and flat JSON input. This `sagemaker-model-monitor-analyzer` container also provides you with a range of model monitoring capabilities, including constraint validation against a baseline, and emitting Amazon CloudWatch metrics. This container is based on Spark and is built with [Deequ](https://github.com/awslabs/deequ). 

All column names in your baseline dataset must be compliant with Spark. For column names, use only lowercase characters, and _ as the only special character.

Use the baseline dataset you created in the step 2 notebook data processing.

In [ ]:
!aws s3 ls {baseline_s3_url}/

In [ ]:
baseline_results_s3_url = f"{baseline_s3_url}/results"
reports_s3_url = f"{baseline_s3_url}/reports"
baseline_dataset_uri = f"{baseline_s3_url}/baseline.csv"
baseline_job_name = f"from-idea-to-prod-processing-baselining-{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"

Start a SageMaker projcessing job on the baseline data to profile data and suggest constraints.

In [ ]:
data_monitor = DefaultModelMonitor(
    role=sm_role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

data_monitor.suggest_baseline(
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_s3_url,
    wait=True,
    logs=False,
    job_name=baseline_job_name,
)

### See the generated statistics and constraints
After the baselining jobs finished, it saves the baseline statistics to the `statistics.json` file and the suggested baseline constraints to the `constraints.json` file in the location you specify with `output_s3_uri`.

In [ ]:
!aws s3 ls {baseline_results_s3_url}/

In [ ]:
statistics_s3_url = f"{baseline_results_s3_url}/statistics.json"
constraints_s3_url = f"{baseline_results_s3_url}/constraints.json"

Load the generated JSON as Pandas DataFrame and see the content of `statistics.json` and `constaints.json`:

In [ ]:
baseline_job = data_monitor.latest_baselining_job
statistics_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
statistics_df.head(59)

In [ ]:
constraints_df = pd.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head()

For this dataset the baselining job suggest three constraints:
1. DataType
2. Completeness
3. Is non-negative

Additionally, the Model Monitor prebuilt container does missing and extra column check, baseline drift check, and categorical values check. Refer to [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-violations.html) for more details.

In a real-world project you can add your own constraints the data must comply with.

Next you schedule and run a monitoring job to validate incoming data against these constraints and statistics.

### Create a monitoring schedule
With a monitoring schedule, SageMaker launches processing jobs at a specified frequency to analyze the data collected during a given period. SageMaker provides a [built-in container](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-built-container.html) for performing analysis on tabular datasets. In the processing job, SageMaker compares the dataset for the current analysis with the baseline statistics and constraints and generates a violations report. In addition, CloudWatch metrics are emitted for each data feature under analysis.

#### Implement custom record processing with a preprocessing script
You can extend Model Monitor by providing a custom record preprocessing function. In this function you can implement your own filtering or preprocessing of every data record. For example, you can skip some records from analysis based on values or some event metadata. Refer to [Preprocessing and Postprocessing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-and-post-processing.html) documentation for more details and examples.

When you created a monitoring baseline, you used the baseline dataset with all features but without the label. The Model Monitor by default concatenates model input and output, resulting in a dataset which contains all features plus the label. If you don't preprocess records before passing them to Model Monitor, the number of columns in the baseline dataset won't match the number of columns in the record, and Model Monitor will report a `extra_column_check` violation. To avoid this situation, you need either to include the label column in the baselining or remove model output from the monitored records. This notebook uses the latter approach and provides a preprocessing script that returns only input data without the label.

For another example of custom preprocessing see the blog post [Design a compelling record filtering method with Amazon SageMaker Model Monitor](https://aws.amazon.com/blogs/machine-learning/design-a-compelling-record-filtering-method-with-amazon-sagemaker-model-monitor/).

In [ ]:
# !pygmentize ./record_preprocessor.py

In [ ]:
# Upload the preprocessing script to S3
record_preprocessor_s3_url = f"s3://{bucket_name}/{bucket_prefix}/code"

In [ ]:
!aws s3 cp ./record_preprocessor.py {record_preprocessor_s3_url}/

In [ ]:
mon_schedule_name = "from-idea-to-prod-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    # post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=reports_s3_url,
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [ ]:
desc_schedule_result = data_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

### Generate compliant traffic
Generate traffic that won't trigger any violations. Use the `test_x.csv` dataset to send requests to the endpoint.

In [ ]:
%%time

generate_endpoint_traffic(predictor, test_x)

### See the captured data
List captured data files under `data_capture_s3_url`. Wait couple of minutes before the captured data appears in the Amazon S3 bucket.

In [ ]:
!aws s3 ls {data_capture_s3_url} --recursive

In [ ]:
#!aws s3 rm {data_capture_s3_url} --recursive

### Launch a manual monitoring job
You can launch a monitoring job manually and don't wait until a configured scheduled execution. Since the Model Monitor uses a [built-in container](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-built-container.html) and a SageMaker [processing job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) to run analysis of the captured data, you can manually configure and run a monitoring job. 

This [repository](https://github.com/aws-samples/reinvent2019-aim362-sagemaker-debugger-model-monitor/tree/master/02_deploy_and_monitor) contains an implementation of a helper function to manually run a monitoring job.

In [ ]:
# !pygmentize ./utils/monitoring_utils.py

Get an S3 url for the latest captured data files:

In [ ]:
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

In [ ]:
print(f"Data capture path: {latest_data_capture_s3_url}")
print(f"Baseline statistics file: {statistics_s3_url}")
print(f"Baseline constraints file: {constraints_s3_url}")
print(f"Data monitor report output path: {reports_s3_url}")
print(f"Record preprocessor script path: {record_preprocessor_s3_url}")

Run a monitoring job:

In [ ]:
utils.monitoring_utils.run_model_monitor_job(
    region=region,
    instance_type="ml.m5.xlarge",
    role=sm_role,
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=statistics_s3_url,
    constraints_path=constraints_s3_url,
    reports_path=reports_s3_url,
    instance_count=1,
    preprocessor_path=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled"
)

### See the monitoring job output
Let's check what reports the monitoring job generated. 

In [ ]:
manual_monitoring_job_output_s3_url = get_latest_monitoring_report_s3_rul("sagemaker-model-monitor-analyzer")

In [ ]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

Load the monitoring reports:

In [ ]:
pd.read_json(f"{manual_monitoring_job_output_s3_url}/constraint_violations.json")

In [ ]:
statistics = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/statistics.json")
constraints = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraints.json")

print(f"Records processed: {statistics['dataset']['item_count']}")

In [ ]:
pd.json_normalize(statistics["features"]).head(59)

In [ ]:
pd.json_normalize(constraints["features"]).head(59)

### Generate non-compliant traffic
Now generate traffic that will trigger the violation in the model monitor data quality check.

In [ ]:
non_compliant_pd = test_x.copy()
non_compliant_pd.iloc[:,0] = -100

In [ ]:
non_compliant_pd.head()

In [ ]:
# Remove previous data capture with compliant traffic
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

In [ ]:
# uncomment the next line to remove the previous data capture files
#!aws s3 rm {latest_data_capture_s3_url} --recursive

In [ ]:
%%time

generate_endpoint_traffic(predictor, non_compliant_pd)

### See the captured data
List captured data files under `data_capture_s3_url`. Wait couple of minutes before the captured data appears in the Amazon S3 bucket.

In [ ]:
!aws s3 ls {data_capture_s3_url} --recursive

### Launch a manual monitoring job
Let's run a manual monitoring job again to analyze the capture data:

In [ ]:
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

In [ ]:
utils.monitoring_utils.run_model_monitor_job(
    region=region,
    instance_type="ml.m5.xlarge",
    role=sm_role,
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=statistics_s3_url,
    constraints_path=constraints_s3_url,
    reports_path=reports_s3_url,
    instance_count=1,
    preprocessor_path=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled"
)

### See the monitoring job output
Let's check what reports the monitoring job generated. Since you send non-compliant data to the endpoint, you must see a violation report.

In [ ]:
manual_monitoring_job_output_s3_url = get_latest_monitoring_report_s3_rul("sagemaker-model-monitor-analyzer")

In [ ]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

Load the monitoring reports:

In [ ]:
pd.read_json(f"{manual_monitoring_job_output_s3_url}/constraint_violations.json")

In [ ]:
statistics = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/statistics.json")
constraints = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraints.json")

print(f"Records processed: {statistics['dataset']['item_count']}")

In [ ]:
pd.json_normalize(statistics["features"]).head(59)

In [ ]:
pd.json_normalize(constraints["features"]).head(59)

### List schedule executions and monitoring reports
You created a hourly schedule above that begins executions on the hour plus 0-20 min buffer. You will have to wait till the clock hit the hour. You can also change the schedule.

This section demonstrates how to work with scheduled monitoring job execution. Python SDK [`DefaultModelMonitor`](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.DefaultModelMonitor) implements helper methods to load and see the executions and monitoring reports.

List executions and view a monitoring job details:

In [ ]:
mon_executions = data_monitor.list_executions()

if len(mon_executions):
    latest_execution = mon_executions[-1]  # get the latest execution
    latest_execution.wait(logs=False)

    print(f"Latest execution status: {latest_execution.describe()['ProcessingJobStatus']}")
    print(f"Latest execution result: {latest_execution.describe()['ExitMessage']}")

    latest_job = latest_execution.describe()
    if latest_job["ProcessingJobStatus"] != "Completed":
        print("No completed executions to inspect further")
    else:
        report_uri = latest_execution.output.destination
        print(f"Report Uri: {report_uri}")
else:
    print("No executions found")

See details about the latest scheduled monitoring execution:

In [ ]:
mon_executions[-1].describe()

Get the latest execution statistics and constraints as objects:

In [ ]:
last_execution_statistics = mon_executions[-1].statistics()
last_execution_violations = mon_executions[-1].constraint_violations()

Load reports into Pandas DataFrame:

In [ ]:
pd.json_normalize(last_execution_statistics.body_dict["features"]).head(59)

In [ ]:
pd.json_normalize(last_execution_violations.body_dict["violations"]).head()

See the baseline and the latest data profiling statistics:

In [ ]:
pd.json_normalize(data_monitor.baseline_statistics().body_dict["features"]).head()

In [ ]:
pd.json_normalize(data_monitor.latest_monitoring_statistics().body_dict["features"]).head()

#### View a violation report
Model monitor outputs any violations compared to the baseline to a violation report.

In [ ]:
!aws s3 ls {reports_s3_url}/ --recursive

In [ ]:
violations = data_monitor.latest_monitoring_constraint_violations()

In [ ]:
if violations:
    pd.set_option("display.max_colwidth", None)
    violations_df = pd.json_normalize(violations.body_dict["violations"])
    violations_df.head()
else:
    print("No violations report found")

In [ ]:
violations_df

---

## Model monitor - monitor model quality
Model quality monitoring jobs monitor the performance of a model by comparing the predictions that the model makes with the actual ground truth labels that the model attempts to predict. To do this, model quality monitoring merges data that is captured from real-time inference with actual labels that you store in an Amazon S3 bucket, and then compares the predictions with the actual labels.

Model quality monitoring follows the same steps as data quality monitoring, but adds the additional step of merging the actual labels from Amazon S3 with the predictions captured from the real-time inference endpoint.

To monitor model quality, follow these steps:
1. Enable [data capture](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html)
1. [Create a baseline](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-baseline.html). A baseline job compares predictions from the model with ground truth labels in a baseline dataset
1. [Schedule monitoring jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-schedule.html)
1. [Ingest ground truth labels](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html) that model monitor merges with captured prediction data from real-time inference endpoint
1. [Intepret the results](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-results.html)
1. [Integrate model quality monitoring](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-cw.html) with Amazon CloudWatch

## Additional monitoring
Additionally to data and model quality monitoring with Model Monitor, you can use Amazon SageMaker Clarify to:
- [Monitor bias drift](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html)
- [Monitor feature attribution drift](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html)

Refer to a sample notebook [Monitoring bias drift and feature attribution drift Amazon SageMaker Clarify](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/fairness_and_explainability/SageMaker-Model-Monitor-Fairness-and-Explainability.html) for a hands-on example and more details.

## Use SageMaker Studio for data and model monitoring
You can use Studio UX to enable and configure data and model monitoring and to visualize results. You can view the details of any monitoring job run, and you can create charts that show the baseline and captured values for any metric that the monitoring job calculates.

Navigate to **SageMaker resources** to the left side bar and choose **Endpoints** in the drop-down menu. Double-click on an endpoint for which you would like to configure the model monitoring:

<img src="img/endpoints.png" width="400"/>

In the displayed **Endpoint details** tab you can configure data and model monitoring:

![](img/model-monitoring-ux.png)

## Clean-up resources

In [ ]:
data_monitor.stop_monitoring_schedule()
data_monitor.delete_monitoring_schedule()

### Final clean-up
This is the last notebook in this workshop. If you are finished with exploration, to avoid charges on your AWS account, run the [clean-up notebook](99-clean-up.ipynb).

## Further development ideas for your real-world projects
- Add [visualizations](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/visualization/SageMaker-Model-Monitor-Visualize.html) for model monitoring reports
- Add data baselining, explainability report generation, and bias report to the model building pipeline
- Implement [model quality monitoring](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html)
- Try different inference options such as [serverless](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html) or [asynchronous](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html) inference
- Address security considerations for your ML environment and solutions. Start with the developer guide [Security in Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html)
- Implement [deployment guardrails](https://docs.aws.amazon.com/sagemaker/latest/dg/deployment-guardrails.html) to control how to update your models in production

## Additional resources
- [AmazonSageMaker Model Monitor: A System for Real-Time Insights into Deployed Machine Learning Models](https://assets.amazon.science/97/cc/8dc8526547859351f46d2710aba9/amazon-sagemaker-model-monitor-a-system-for-real-time-insights-into-deployed-machine-learning-models.pdf)
- [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)
- [Monitor data quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-quality.html)
- [Model Monitor visualizations](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/visualization/SageMaker-Model-Monitor-Visualize.html)
- [Monitor Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-overview.html)
- [Monitoring a Model in Production](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-model-monitor.html)
- [ModelMonitor for batch transform jobs](https://aws.amazon.com/about-aws/whats-new/2022/10/amazon-sagemaker-model-monitor-batch-transform-jobs/)
- [Security in Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html)
- [Deployment guardrails](https://docs.aws.amazon.com/sagemaker/latest/dg/deployment-guardrails.html)
- [Design a compelling record filtering method with Amazon SageMaker Model Monitor](https://aws.amazon.com/blogs/machine-learning/design-a-compelling-record-filtering-method-with-amazon-sagemaker-model-monitor/)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>